<a href="https://colab.research.google.com/github/heraclex12/ISODS-Conversational-Sentiment-Analysis/blob/main/Conversation_TF_IDF_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install vncorenlp

     |████████████████████████████████| 2.7MB 1.4MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645934 sha256=699081ad7a1f8801c4babb13f66c97858b5e0e60f3464472870d27e20df9f8cf
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


In [ ]:
import os

from sklearn.metrics import accuracy_score, f1_score
from vncorenlp import VnCoreNLP
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
import csv
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
import pickle
import scipy.sparse
pd.set_option('display.max_colwidth', -1)

In [17]:
filename = 'test.csv'

In [ ]:
df_test = pd.read_csv(filename)

In [ ]:
import re
import string


data_test_q = df_test.question.apply(lambda s: re.sub('[' + string.punctuation + ']', ' ', s)).values
data_test_a = df_test.answer.apply(lambda s: re.sub('[' + string.punctuation + ']', ' ', s)).values

def transform_text(X):
    conversations = [". ".join(messages.lower().split('|||')) for messages in X]
    conversations = [" ".join(" ".join(sentence) for sentence in tokenizer.tokenize(message)) for message in conversations]
    return conversations

tokenizer = VnCoreNLP('VnCoreNLP-1.1.1.jar', annotators='wseg')
data_test_q = transform_text(data_test_q)
data_test_a = transform_text(data_test_a)

In [ ]:
model = pickle.load(open('model.pkl', 'rb'))
vectorize_a = pickle.load(open('vectorize_a.pkl', 'rb'))
vectorize_q = pickle.load(open('vectorize_q.pkl', 'rb'))

In [ ]:
test_vector_q = vectorize_q.transform(data_test_q)
test_vector_a = vectorize_a.transform(data_test_a)

test_vector = scipy.sparse.hstack([test_vector_q, test_vector_a])

In [ ]:
preds = model.predict_proba(test_vector)[:, 1]
preds
with open('submission.csv', 'w') as csv_file:
  writer = csv.writer(csv_file)
  writer.writerow(['num', 'label'])
  pred_label = 1*(preds>=0.8)
  for i, label in enumerate(pred_label):
    writer.writerow([i + 1, 'Y' if label == 1 else 'N'])